## Load Data

In [ ]:
import pandas as pd
import numpy as numpy

In [ ]:
import os
a = os.getcwd()
print(a)

/content


In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [ ]:
ls

drive/  sample_data/


In [ ]:

import os

os.chdir("/content/drive/Shared drives/Praxis_Analysis")

In [ ]:
ls

 Base_Models.ipynb            Final_Data.csv   SpamAssasin.csv      TREC_07.gsheet
'Data_Analysis- LSTM.ipynb'   Nazario.csv      SpamAssasin.gsheet
 Final_Clean_Data.csv         Nazario.gsheet   TREC_07.csv


In [ ]:
# df1 = pd.read_csv("TREC_05.csv")
# df2 = pd.read_csv("TREC_06.csv")
# df1 = pd.read_csv("Nazario.csv")
# df2 = pd.read_csv("SpamAssasin.csv")
# df3 = pd.read_csv("TREC_07.csv")

In [ ]:
# print(df1.head().to_string())
# print(df2.head().to_string())
# print(df3.head().to_string())

In [ ]:
#print(df1.columns)

In [ ]:
#print(df2.columns)

In [ ]:
#print(df3.columns)

In [ ]:
# print(len(df1))
# print(len(df2))
# print(len(df3))

In [ ]:
# df1['url'] = df1["urls"]
# df2['url'] = df2["urls"]
# df3['url'] = df3["urls"]

In [ ]:
# print(df1.columns)
# print(df2.columns)
# print(df3.columns)

In [ ]:
# df1 = df1.drop(columns=['urls'])
# df2 = df2.drop(columns=['urls'])
# df3 = df3.drop(columns=['urls'])

In [ ]:
# print(df1.columns)
# print(df2.columns)
# print(df3.columns)

In [ ]:
#df = pd.concat([df1, df2, df3], ignore_index=True, axis=0)

In [ ]:
#df.head()

In [ ]:
#print(len(df))

In [ ]:
#df.to_csv("Final_Data.csv")

In [ ]:
#data = pd.read_csv("Final_Data.csv")

In [ ]:
#print(df.isna().sum())

In [ ]:
#df = df.drop(columns=['receiver','date'])

In [ ]:
#print(df.isna().sum())

In [ ]:
#df = df.dropna(axis=0)

In [ ]:
#print(df.isna().sum())

In [ ]:
#len(df)

In [ ]:
#df = df.drop_duplicates(keep='first') # check for duplicates

In [ ]:
#len(df)

In [ ]:
#df["email_content"] = df["subject"]+ " " + df["body"]+ " " +df["sender"]

In [ ]:
#df = df.drop(columns=["subject","body","url","sender"])

In [ ]:
#df

In [ ]:
#df['label'] = df['label'].map({1:'phishing',0:'spam'})

In [ ]:
#df

In [ ]:
#df.to_csv("Final_Clean_Data.csv")

In [ ]:
df = pd.read_csv("Final_Clean_Data.csv")

In [ ]:
# import sklearn as sk
# from sklearn.model_selection import train_test_split

In [ ]:
# train , valtest = train_test_split(df,test_size=.3,shuffle= True)

In [ ]:
# val , test = train_test_split(valtest,test_size=.5,shuffle= True)

In [ ]:
# print(len(train))
# print(len(test))
# print(len(val))

In [ ]:
# Xtrain = train.drop(columns=["label"])
# ytrain = train[['label']]
# Xtest = test.drop(columns=["label"])
# ytest = test[['label']]
# Xval = val.drop(columns=["label"])
# yval = val[['label']]

In [ ]:
#Xtrain.head()

In [ ]:
# ytrain.value_counts()

In [ ]:
df.head()

,Unnamed: 0,label,email_content
0,0,phishing,DON'T DELETE THIS MESSAGE -- FOLDER INTERNAL D...
1,1,phishing,Verify Your Account Business with \t\t\t\t\t\...
2,2,phishing,Helpdesk Mailbox Alert!!! Your two incoming ma...
3,3,phishing,IT-Service Help Desk Password will expire in 3...
4,4,phishing,Final USAA Reminder - Update Your Account Now ...


Now comes Data Preprocessing. Here we will do the following 1. change data to lowecase 2. take out stop words 3. stemming and lemmtization 4. vectorize the data followed by tokenization.


## Define Dataset Class

In [ ]:
%%capture
!pip install transformers
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel, GPT2Tokenizer
from torch import nn
from torch.optim import Adam
from tqdm import tqdm


import torch
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from collections import Counter

# Dataset
from torch.utils.data import Dataset

### Conventional Dataset

In [ ]:
# Example vocabulary dictionary: word to index mapping
# In a real scenario, this should be built from your training data
# Tokenize each email and flatten the list of tokens
all_tokens = [word_tokenize(email.lower()) for email in df['email_content']]
flattened_tokens = [token for sublist in all_tokens for token in sublist]

# Identify unique tokens and their frequencies
token_counts = Counter(flattened_tokens)

# Extract just the tokens to form the vocabulary
vocab = list(token_counts.keys())
special_tokens = ['[PAD]', '[UNK]']
vocab = special_tokens + vocab

print(len(vocab))
# Assuming 'filtered_vocab' is your list of vocabulary words including special tokens
vocab = {word: i for i, word in enumerate(vocab)}

# Function to convert tokens to IDs using the vocabulary
def tokens_to_ids(tokens, vocab, max_length):
    ids = [vocab.get(token, vocab['[UNK]']) for token in tokens][:max_length]
    masks = [1]*len(ids) + [0]*(max_length - len(ids)) # mask values
    ids += [vocab['[PAD]']] * (max_length - len(ids))  # Padding
    return ids, masks

626489


In [ ]:
labels = {'spam': 0, 'phishing': 1}

class SpamDataset(torch.utils.data.Dataset):
    def __init__(self, df, vocab=vocab, max_length=512):
        self.vocab = vocab
        self.max_length = max_length
        self.labels = [labels[label] for label in df['label']]
        self.texts = [word_tokenize(text) for text in df['email_content']]
        self.tokens = [tokens_to_ids(text, vocab, max_length) for text in self.texts]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        y = np.array(self.labels[idx])
        ids, masks = self.tokens[idx]

        return torch.tensor(ids), torch.tensor(masks), y

### Bert Dataset


In [ ]:
# Note: The `vocab` should be defined according to your dataset.
# The `[PAD]` and `[UNK]` tokens are used for padding and unknown words, respectively.


# from transformers import GPT2Tokenizer

# model_name_or_path = 'gpt2'

# tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path=model_name_or_path)
# if tokenizer.pad_token is None:
    # Set the EOS token as the pad token
#    tokenizer.pad_token = tokenizer.eos_token
labels = {'spam':0,'phishing':1}

class SpamDatasetBert(torch.utils.data.Dataset):

    def __init__(self, df):
        tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
        self.labels = [labels[label] for label in df['label']]
        self.tokens = [tokenizer(text,
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['email_content']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        ids, masks = self.tokens[idx]['input_ids'][0], self.tokens[idx]['attention_mask'][0]
        batch_y = np.array(self.labels[idx])
        return ids, masks, batch_y

### GPT2 Dataset


In [ ]:

labels = {'spam':0,'phishing':1}

class SpamDatasetGpt(torch.utils.data.Dataset):

    def __init__(self, df):

        tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path='gpt2')
        # Set the EOS token as the pad token
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        self.labels = [labels[label] for label in df['label']]
        self.tokens = [tokenizer(text,
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['email_content']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        ids, masks = self.tokens[idx]['input_ids'][0], self.tokens[idx]['attention_mask'][0]
        batch_y = np.array(self.labels[idx])
        return ids, masks, batch_y

## Define Models

### MLP4/MLP8/CNN/LSTM/CNN-LSTM Models

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch
import torch.nn as nn
import torch.nn.functional as F
class LstmClassifier(nn.Module):
    def __init__(self, vocab_size=626489, embedding_dim=128, lstm_hidden_size=256, dropout=0.5):
       super(LstmClassifier, self).__init__()

       # Embedding layer that converts input word indices to embeddings
       self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)


       # Define the LSTM layer
       self.lstm = nn.LSTM(input_size=embedding_dim,  # Adjusted for embedding dimension
                           hidden_size=lstm_hidden_size,
                           num_layers=1,
                           bidirectional=False,
                           batch_first=True)

       # Dropout layer
       self.dropout = nn.Dropout(dropout)


       # Fully connected layer that outputs raw scores for each class
       self.linear = nn.Linear(lstm_hidden_size, 1)  # Assuming binary classification


       # Sigmoid activation layer
       self.sigmoid = nn.Sigmoid()


    def forward(self, input_ids, attention_mask = None):
        # Word indices to embeddings
        embeddings = self.embedding(input_ids)

        # Calculate lengths from attention_mask if provided, else assume no padding
        if attention_mask is not None:
            lengths = attention_mask.sum(dim = 1)
        else:
            lengths = input_ids.ne(0).sum(dim = 1)

        # Sort batches by lengths in descending order for pack_padded_sequence
        lengths, sorted_idx = lengths.sort(descending = True)
        embeddings = embeddings[sorted_idx]
        packed_embeddings = pack_padded_sequence(embeddings, lengths.cpu(), batch_first = True)

        # Process thru LSTM
        packed_lstm_output, (hidden, cell) = self.lstm(packed_embeddings)
        lstm_output, _ = pad_packed_sequence(packed_lstm_output, batch_first = True)

        # Revert the sorting
        _, reversed_idx = sorted_idx.sort(descending = False)
        lstm_output = lstm_output[reversed_idx]

        # Use lengths to select the final output for each sequence
        final_output = lstm_output[torch.arange(lstm_output.size(0)), lengths -1]

        # Apply dropout
        dropout_output = self.dropout(final_output)

        # Pass thru the linear layer and then sigmoid activtion fn
        final_layer = self.sigmoid(self.linear(dropout_output))

        return final_layer


class CnnClassifier(nn.Module):
    def __init__(self, vocab_size=626489, embedding_dim=128, cnn_out_channels=256, dropout=0.5):
        super(CnnClassifier, self).__init__()

        # Embedding layer that converts input word indices to embeddings
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

        # Define the 1D CNN layer
        self.conv1d = nn.Conv1d(in_channels=embedding_dim,  # Input channels equal to embedding dimension
                                out_channels=cnn_out_channels,  # Output channels
                                kernel_size=5,  # Kernel size
                                stride=1,  # Stride
                                padding=2)  # Padding to keep the sequence length unchanged

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

        # Fully connected layer that outputs raw scores for each class
        self.linear = nn.Linear(cnn_out_channels, 1)  # Assuming binary classification

        # Sigmoid activation layer
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask=None):
        # Word indices to embeddings
        embeddings = self.embedding(input_ids)

        # Transpose batch and embedding dimensions: [batch_size, seq_len, emb_dim] to [batch_size, emb_dim, seq_len]
        embeddings_transposed = embeddings.transpose(1, 2)

        # Apply 1D convolution
        conv_output = self.conv1d(embeddings_transposed)

        # Apply non-linearity (e.g., ReLU)
        conv_output = F.relu(conv_output)

        # Global max pooling over the outputs
        pooled_output, _ = torch.max(conv_output, 2)

        # Apply dropout
        dropout_output = self.dropout(pooled_output)

        # Pass through the linear layer and then sigmoid activation function
        final_layer = self.sigmoid(self.linear(dropout_output))

        return final_layer


class CnnLstmClassifier(nn.Module):
    def __init__(self, vocab_size=626489, embedding_dim=128, cnn_out_channels=256, lstm_hidden_size=256, dropout=0.5):
        super(CnnLstmClassifier, self).__init__()

        # Embedding layer that converts input word indices to embeddings
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

        # Define the 1D CNN layer
        self.conv1d = nn.Conv1d(in_channels=embedding_dim,
                                out_channels=cnn_out_channels,
                                kernel_size=5,
                                stride=1,
                                padding=2)  # Padding to keep the sequence length unchanged

        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size=cnn_out_channels,  # Adjusted for the output of CNN
                            hidden_size=lstm_hidden_size,
                            num_layers=1,
                            bidirectional=False,
                            batch_first=True)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

        # Fully connected layer that outputs raw scores for each class
        self.linear = nn.Linear(lstm_hidden_size, 1)  # Assuming binary classification

        # Sigmoid activation layer
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask=None):
        # Word indices to embeddings
        embeddings = self.embedding(input_ids)

        # Transpose batch and embedding dimensions: [batch_size, seq_len, emb_dim] to [batch_size, emb_dim, seq_len]
        embeddings_transposed = embeddings.transpose(1, 2)

        # Apply 1D convolution
        conv_output = self.conv1d(embeddings_transposed)

        # Apply non-linearity (e.g., ReLU)
        conv_output = F.relu(conv_output)

        # Transpose back to match LSTM input shape: [batch_size, seq_len, cnn_out_channels]
        conv_output_transposed = conv_output.transpose(1, 2)

        # Process through LSTM
        lstm_output, (hidden, cell) = self.lstm(conv_output_transposed)

        # Apply dropout to the last hidden state
        dropout_output = self.dropout(hidden[-1])

        # Pass through the linear layer and then sigmoid activation function
        final_layer = self.sigmoid(self.linear(dropout_output))

        return final_layer

import torch
import torch.nn as nn

class Mlp4LayerClassifier(nn.Module):
    def __init__(self, vocab_size=626489, embedding_dim=128, mlp_hidden_size=256, dropout=0.5):
        super(Mlp4LayerClassifier, self).__init__()

        # Embedding layer
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

        # MLP layers
        self.fc1 = nn.Linear(embedding_dim, mlp_hidden_size)
        self.fc2 = nn.Linear(mlp_hidden_size, mlp_hidden_size // 2)
        self.fc3 = nn.Linear(mlp_hidden_size // 2, mlp_hidden_size // 4)
        self.fc4 = nn.Linear(mlp_hidden_size // 4, 1)  # Output layer

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

        # Activation functions
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask=None):
        # Embedding layer
        embeddings = self.embedding(input_ids)

        # Average embeddings along the sequence length dimension
        avg_embeddings = embeddings.mean(dim=1)

        # MLP forward pass
        x = self.relu(self.fc1(avg_embeddings))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc4(x))

        return x


class Mlp8LayerClassifier(nn.Module):
    def __init__(self, vocab_size=626489, embedding_dim=128, mlp_hidden_size=256, dropout=0.5):
        super(Mlp8LayerClassifier, self).__init__()

        # Embedding layer
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

        # MLP layers
        self.fc1 = nn.Linear(embedding_dim, mlp_hidden_size)
        self.fc2 = nn.Linear(mlp_hidden_size, mlp_hidden_size)
        self.fc3 = nn.Linear(mlp_hidden_size, mlp_hidden_size // 2)
        self.fc4 = nn.Linear(mlp_hidden_size // 2, mlp_hidden_size // 2)
        self.fc5 = nn.Linear(mlp_hidden_size // 2, mlp_hidden_size // 4)
        self.fc6 = nn.Linear(mlp_hidden_size // 4, mlp_hidden_size // 4)
        self.fc7 = nn.Linear(mlp_hidden_size // 4, mlp_hidden_size // 8)
        self.fc8 = nn.Linear(mlp_hidden_size // 8, 1)  # Output layer

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

        # Activation functions
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask=None):
        # Embedding layer
        embeddings = self.embedding(input_ids)

        # Average embeddings along the sequence length dimension
        avg_embeddings = embeddings.mean(dim=1)

        # MLP forward pass
        x = self.relu(self.fc1(avg_embeddings))
        x = self.dropout(x)
        for fc in [self.fc2, self.fc3, self.fc4, self.fc5, self.fc6, self.fc7]:
            x = self.relu(fc(x))
            x = self.dropout(x)
        x = self.sigmoid(self.fc8(x))

        return x



### BERT Models


In [ ]:
from transformers import BertModel
import torch.nn as nn
class BertCNNLstmClassifier(nn.Module):
    def __init__(self, dropout=0.5, lstm_hidden_size=256, conv_out_channels=128, kernel_size=3):
        super(BertCNNLstmClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')

        # Define a 1D Convolutional layer to apply on BERT output
        self.conv1d = nn.Conv1d(in_channels=768,  # BERT embedding size
                                out_channels=conv_out_channels,
                                kernel_size=kernel_size,
                                padding='same')  # This will keep the sequence length the same

        self.lstm = nn.LSTM(input_size=conv_out_channels,  # Adjusted for output of conv1d
                            hidden_size=lstm_hidden_size,
                            num_layers=1,
                            bidirectional=False,
                            batch_first=True)

        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(lstm_hidden_size, 1)  # Adjust linear layer input size
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):
        # Get BERT output
        bert_output = self.bert(input_ids=input_id, attention_mask=mask).last_hidden_state

        # Reshape for CNN: [batch_size, channels, length]
        cnn_input = bert_output.transpose(1, 2)  # Transpose to [batch_size, 768, seq_len]

        # Process through 1D Convolution
        cnn_output = self.conv1d(cnn_input)

        # Transpose back to [batch_size, seq_len, features] for LSTM
        lstm_input = cnn_output.transpose(1, 2)

        # Process through LSTM
        lstm_output, _ = self.lstm(lstm_input)

        # Assuming you want the final LSTM output for classification
        final_output = lstm_output[:, -1, :]  # Take last time step

        dropout_output = self.dropout(final_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigmoid(linear_output)

        return final_layer


class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigmoid(linear_output)

        return final_layer

from transformers import BertModel
import torch.nn as nn

class BertLstmClassifier(nn.Module):
    def __init__(self, dropout=0.5, lstm_hidden_size=256):
        super(BertLstmClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.lstm = nn.LSTM(input_size=768,
                            hidden_size=lstm_hidden_size,
                            num_layers=1,
                            bidirectional=False,
                            batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(lstm_hidden_size, 1)  # Change input size to match LSTM hidden size
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False)

        # Reshape pooled_output for LSTM: [batch_size, 1, 768]
        pooled_output = pooled_output.unsqueeze(1)

        # Process through LSTM
        lstm_output, _ = self.lstm(pooled_output)

        # We take the output from the last LSTM cell
        lstm_output = lstm_output.squeeze(1)  # [batch_size, lstm_hidden_size]

        dropout_output = self.dropout(lstm_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigmoid(linear_output)

        return final_layer

from transformers import BertModel
import torch.nn as nn

class BertCNNClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(BertCNNClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.conv1d = nn.Conv1d(in_channels=768, out_channels=256, kernel_size=1)  # Example configuration
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(256, 1)  # Adjust input size to match Conv1d output
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False)

        # Reshape pooled_output for Conv1d: [batch_size, in_channels, 1]
        pooled_output = pooled_output.unsqueeze(2)

        # Process through Conv1d
        conv_output = self.conv1d(pooled_output)

        # Reshape conv_output to remove the last dimension: [batch_size, out_channels]
        conv_output = conv_output.squeeze(2)

        dropout_output = self.dropout(conv_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigmoid(linear_output)

        return final_layer



### GPT2 Models

In [ ]:
from transformers import GPT2Model

class GPT2Classifier(nn.Module):

    def __init__(self, dropout=0.5):
        super(GPT2Classifier, self).__init__()

        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        self.dropout = nn.Dropout(dropout)
        # GPT-2 also has a hidden size of 768 for the base model
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):
        # Get outputs from GPT-2 model. GPT-2 does not use a 'mask' in the same way BERT does for attention,
        # but it does have an 'attention_mask' parameter for ignoring padding tokens.
        outputs = self.gpt2(input_ids=input_id, attention_mask=mask)
        # Use the last hidden state
        last_hidden_state = outputs.last_hidden_state
        # You can use the last token's hidden state or mean pooling as a representation.
        # Here, we use the last token's hidden state for classification.
        # This is a simplification; other strategies might involve pooling across all tokens.
        last_token_state = last_hidden_state[:, -1, :]  # Take the last token for each sequence
        dropout_output = self.dropout(last_token_state)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigmoid(linear_output)

        return final_layer


from transformers import BertModel
import torch.nn as nn

class GPT2CNNClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super(GPT2CNNClassifier, self).__init__()

        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        self.dropout = nn.Dropout(dropout)

        # Add a 1D CNN layer with appropriate parameters
        self.cnn1d = nn.Conv1d(in_channels=768, out_channels=64, kernel_size=3, padding=1)

        # GPT-2 has a hidden size of 768 for the base model
        self.linear = nn.Linear(64, 1)  # Update the input size for the linear layer
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):
        # Get outputs from GPT-2 model
        outputs = self.gpt2(input_ids=input_id, attention_mask=mask)

        # Extract the last hidden state
        last_hidden_state = outputs.last_hidden_state

        # Select the last token's hidden state
        last_token_state = last_hidden_state[:, -1, :]

        # Apply dropout
        dropout_output = self.dropout(last_token_state)

        # Transpose the tensor for 1D CNN (B, C, L) -> (B, L, C)
        cnn_input = dropout_output.unsqueeze(1).permute(0, 2, 1)

        # Apply 1D CNN
        cnn_output = self.cnn1d(cnn_input)

        # Reshape the CNN output
        cnn_output = cnn_output.view(cnn_output.size(0), -1)

        # Apply linear layer
        linear_output = self.linear(cnn_output)

        # Apply sigmoid activation
        final_layer = self.sigmoid(linear_output)

        return final_layer



class GPT2LstmClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super(GPT2LstmClassifier, self).__init__()

        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        self.dropout = nn.Dropout(dropout)

        # GPT-2 also has a hidden size of 768 for the base model
        self.lstm = nn.LSTM(input_size=768, hidden_size=128, num_layers=1, batch_first=True, bidirectional=False)  # Add a unidirectional LSTM layer

        self.linear = nn.Linear(128, 1)  # Update the input size for the linear layer
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):
        # Get outputs from GPT-2 model
        outputs = self.gpt2(input_ids=input_id, attention_mask=mask)

        # Extract the last hidden state
        last_hidden_state = outputs.last_hidden_state

        # Select the last token's hidden state
        last_token_state = last_hidden_state[:, -1, :]

        # Apply dropout
        dropout_output = self.dropout(last_token_state)

        # Apply LSTM
        lstm_output, _ = self.lstm(dropout_output.unsqueeze(1))  # LSTM expects 3D input

        # Select the last time step's output from the LSTM
        lstm_last_output = lstm_output[:, -1, :]

        # Apply linear layer
        linear_output = self.linear(lstm_last_output)

        # Apply sigmoid activation
        final_layer = self.sigmoid(linear_output)

        return final_layer



import torch.nn as nn
from transformers import GPT2Model

class GPT2CNNLstmClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super(GPT2CNNLstmClassifier, self).__init__()

        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        self.dropout = nn.Dropout(dropout)

        # GPT-2 also has a hidden size of 768 for the base model
        self.cnn1d = nn.Conv1d(in_channels=768, out_channels=64, kernel_size=3, padding=1)  # Add a 1D CNN layer
        self.lstm = nn.LSTM(input_size=64, hidden_size=128, num_layers=1, batch_first=True, bidirectional=False)  # Add a unidirectional LSTM layer

        self.linear = nn.Linear(128, 1)  # Update the input size for the linear layer
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):
        # Get outputs from GPT-2 model
        outputs = self.gpt2(input_ids=input_id, attention_mask=mask)

        # Extract the last hidden state
        last_hidden_state = outputs.last_hidden_state

        # Select the last token's hidden state
        last_token_state = last_hidden_state[:, -1, :]

        # Apply dropout
        dropout_output = self.dropout(last_token_state)

        # Transpose the tensor for 1D CNN (B, L, C) -> (B, C, L)
        cnn_input = dropout_output.unsqueeze(1).permute(0, 2, 1)

        # Apply 1D CNN
        cnn_output = self.cnn1d(cnn_input)

        # Reshape the CNN output
        cnn_output = cnn_output.view(cnn_output.size(0), -1)

        # Apply LSTM
        lstm_output, _ = self.lstm(cnn_output.unsqueeze(1))  # LSTM expects 3D input

        # Select the last time step's output from the LSTM
        lstm_last_output = lstm_output[:, -1, :]

        # Apply linear layer
        linear_output = self.linear(lstm_last_output)

        # Apply sigmoid activation
        final_layer = self.sigmoid(linear_output)

        return final_layer





## Define Traing/Validation Loops

In [ ]:
def train(model, train_data, val_data, learning_rate, epochs, batch_size=16):

    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.BCELoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_mask, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device).float()
                train_input = train_input.to(device)
                train_mask = train_mask.to(device)

                output = model(train_input, train_mask).squeeze()
                #,mask

                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                predicted = (output>0.5).float()

                acc = (predicted == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_mask, val_label in val_dataloader:

                    val_label = val_label.to(device).float()
                    val_input = val_input.to(device)
                    val_mask = val_mask.to(device)

                    output = model(val_input, val_mask).squeeze()

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    predicted = (output>0.5).float()

                    acc = (predicted == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')


In [ ]:
# def train(model, train_data, val_data, learning_rate, epochs):

#     train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=20, shuffle=True)
#     val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=20)

#     use_cuda = torch.cuda.is_available()
#     device = torch.device("cuda" if use_cuda else "cpu")

#     criterion = nn.BCELoss()
#     optimizer = Adam(model.parameters(), lr= learning_rate)

#     if use_cuda:

#             model = model.cuda()
#             criterion = criterion.cuda()

#     for epoch_num in range(epochs):

#             total_acc_train = 0
#             total_loss_train = 0

#             for train_input, train_label in tqdm(train_dataloader):

#                 train_label = train_label.to(device).float()
#                 train_input = train_input['input_ids'].squeeze(1).to(device)
#                 train_mask = train_input['attention_mask'].to(device)

#                 output = model(train_input, train_mask).squeeze()

#                 batch_loss = criterion(output, train_label)
#                 total_loss_train += batch_loss.item()
#                 predicted = (output>0.5).float()

#                 acc = (predicted == train_label).sum().item()
#                 total_acc_train += acc

#                 model.zero_grad()
#                 batch_loss.backward()
#                 optimizer.step()

#             total_acc_val = 0
#             total_loss_val = 0

#             with torch.no_grad():

#                 for val_input, val_label in val_dataloader:

#                     val_label = val_label.to(device).float()
#                     val_input = val_input['input_ids'].squeeze(1).to(device)
#                     val_mask = val_input['attention_mask'].to(device)

#                     output = model(val_input, val_mask).squeeze()

#                     batch_loss = criterion(output, val_label)
#                     total_loss_val += batch_loss.item()
#                     predicted = (output>0.5).float()

#                     acc = (predicted == val_label).sum().item()
#                     total_acc_val += acc

#             print(
#                 f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')


In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device).float()
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask).squeeze()

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

## Train/Test Split

In [ ]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

48528 6066 6067


## Train Model & Results

In [ ]:
# train_data = SpamDataset(df_train.head())
# train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=20, shuffle=True)
# train_input, train_mask, train_label  = next(iter(train_dataloader))
# train_input = train_input.cuda()
# train_mask = train_mask.cuda()

In [ ]:
train_data = SpamDatasetBert(df_train.head())
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=20, shuffle=True)
train_input, train_mask, train_label  = next(iter(train_dataloader))
model = BertLstmClassifier()

NameError: name 'SpamDatasetBert' is not defined

In [ ]:
# train_input.shape, train_mask.shape

In [ ]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [ ]:
# inputs = tokenizer('mu dog is cute', return_tensors='pt')

In [ ]:
# for key in inputs:
#     inputs[key] = inputs[key].cuda()

In [ ]:
# model.bert(**inputs)

In [ ]:
# train_input

In [ ]:
# bert_output = model.bert(input_ids=train_input, attention_mask=train_mask).last_hidden_state

In [ ]:
# model(train_input, train_mask)

In [ ]:
# df_train['label'].value_counts(1)

### Conventional Models - 1 EPOCHS


In [ ]:
train_data, val_data = SpamDataset(df_train), SpamDataset(df_val)

In [ ]:
EPOCHS = 1
model = Mlp4LayerClassifier()
LR = 1e-4

train(model,train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [00:51<00:00, 59.05it/s]


Epochs: 1 | Train Loss:  0.029 | Train Accuracy:  0.772 | Val Loss:  0.015 | Val Accuracy:  0.911


In [ ]:
EPOCHS = 1
model = Mlp8LayerClassifier()
LR = 1e-4

train(model,train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [00:53<00:00, 56.51it/s]


Epochs: 1 | Train Loss:  0.035 | Train Accuracy:  0.711 | Val Loss:  0.019 | Val Accuracy:  0.909


In [ ]:
EPOCHS = 1
model = LstmClassifier()
LR = 1e-4

train(model,train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [03:44<00:00, 13.51it/s]


Epochs: 1 | Train Loss:  0.037 | Train Accuracy:  0.690 | Val Loss:  0.030 | Val Accuracy:  0.776


In [ ]:
EPOCHS = 1
model = CnnClassifier()
LR = 1e-4

train(model,train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [00:52<00:00, 57.56it/s]


Epochs: 1 | Train Loss:  0.015 | Train Accuracy:  0.901 | Val Loss:  0.008 | Val Accuracy:  0.955


In [ ]:
EPOCHS = 1
model = CnnLstmClassifier()
LR = 1e-4

train(model,train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [03:09<00:00, 16.01it/s]


Epochs: 1 | Train Loss:  0.041 | Train Accuracy:  0.602 | Val Loss:  0.042 | Val Accuracy:  0.576


### Conventional Models - 3 EPOCHS


In [ ]:
train_data, val_data = SpamDataset(df_train), SpamDataset(df_val)

In [ ]:
EPOCHS = 3
model = Mlp4LayerClassifier()
LR = 1e-4

train(model,train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [00:50<00:00, 59.76it/s]


Epochs: 1 | Train Loss:  0.028 | Train Accuracy:  0.779 | Val Loss:  0.017 | Val Accuracy:  0.896


100%|██████████| 3033/3033 [00:50<00:00, 59.64it/s]


Epochs: 2 | Train Loss:  0.012 | Train Accuracy:  0.930 | Val Loss:  0.011 | Val Accuracy:  0.937


100%|██████████| 3033/3033 [00:50<00:00, 60.13it/s]


Epochs: 3 | Train Loss:  0.009 | Train Accuracy:  0.949 | Val Loss:  0.010 | Val Accuracy:  0.956


In [ ]:
EPOCHS = 3
model = Mlp8LayerClassifier()
LR = 1e-4

train(model,train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [00:55<00:00, 54.98it/s]


Epochs: 1 | Train Loss:  0.038 | Train Accuracy:  0.674 | Val Loss:  0.021 | Val Accuracy:  0.884


100%|██████████| 3033/3033 [00:56<00:00, 53.26it/s]


Epochs: 2 | Train Loss:  0.016 | Train Accuracy:  0.919 | Val Loss:  0.013 | Val Accuracy:  0.938


100%|██████████| 3033/3033 [00:55<00:00, 54.29it/s]


Epochs: 3 | Train Loss:  0.012 | Train Accuracy:  0.943 | Val Loss:  0.011 | Val Accuracy:  0.949


In [ ]:
EPOCHS = 3
model = LstmClassifier()
LR = 1e-4

train(model,train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [03:52<00:00, 13.03it/s]


Epochs: 1 | Train Loss:  0.037 | Train Accuracy:  0.673 | Val Loss:  0.035 | Val Accuracy:  0.740


100%|██████████| 3033/3033 [03:51<00:00, 13.10it/s]


Epochs: 2 | Train Loss:  0.032 | Train Accuracy:  0.745 | Val Loss:  0.031 | Val Accuracy:  0.766


100%|██████████| 3033/3033 [03:51<00:00, 13.10it/s]


Epochs: 3 | Train Loss:  0.030 | Train Accuracy:  0.760 | Val Loss:  0.028 | Val Accuracy:  0.788


In [ ]:
EPOCHS = 3
model = CnnClassifier()
LR = 1e-4

train(model,train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [00:53<00:00, 56.52it/s]


Epochs: 1 | Train Loss:  0.014 | Train Accuracy:  0.903 | Val Loss:  0.008 | Val Accuracy:  0.959


100%|██████████| 3033/3033 [00:53<00:00, 56.97it/s]


Epochs: 2 | Train Loss:  0.007 | Train Accuracy:  0.963 | Val Loss:  0.006 | Val Accuracy:  0.968


100%|██████████| 3033/3033 [00:54<00:00, 56.09it/s]


Epochs: 3 | Train Loss:  0.005 | Train Accuracy:  0.973 | Val Loss:  0.005 | Val Accuracy:  0.973


In [ ]:
EPOCHS = 3
model = CnnLstmClassifier()
LR = 1e-4

train(model,train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [03:10<00:00, 15.89it/s]


Epochs: 1 | Train Loss:  0.041 | Train Accuracy:  0.603 | Val Loss:  0.037 | Val Accuracy:  0.611


100%|██████████| 3033/3033 [03:10<00:00, 15.93it/s]


Epochs: 2 | Train Loss:  0.027 | Train Accuracy:  0.791 | Val Loss:  0.011 | Val Accuracy:  0.945


100%|██████████| 3033/3033 [03:12<00:00, 15.79it/s]


Epochs: 3 | Train Loss:  0.011 | Train Accuracy:  0.945 | Val Loss:  0.068 | Val Accuracy:  0.719


### BERT Models - 1 EPOCHS

In [ ]:
train_data, val_data = SpamDatasetBert(df_train), SpamDatasetBert(df_val)

In [ ]:
EPOCHS = 1
model = BertClassifier()
LR = 1e-4

train(model, train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [24:22<00:00,  2.07it/s]


Epochs: 1 | Train Loss:  0.041 | Train Accuracy:  0.560 | Val Loss:  0.045 | Val Accuracy:  0.504


In [ ]:
EPOCHS = 1
model = BertLstmClassifier()
LR = 1e-4

train(model, train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [24:37<00:00,  2.05it/s]


Epochs: 1 | Train Loss:  0.041 | Train Accuracy:  0.576 | Val Loss:  0.043 | Val Accuracy:  0.534


In [ ]:
EPOCHS = 1
model = BertCNNLstmClassifier()
LR = 1e-4

train(model, train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [28:05<00:00,  1.80it/s]


Epochs: 1 | Train Loss:  0.042 | Train Accuracy:  0.544 | Val Loss:  0.043 | Val Accuracy:  0.535


In [ ]:
EPOCHS = 1
model = BertCNNClassifier()
LR = 1e-4

train(model, train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [24:29<00:00,  2.06it/s]


Epochs: 1 | Train Loss:  0.039 | Train Accuracy:  0.578 | Val Loss:  0.045 | Val Accuracy:  0.535


### BERT Models - 3 EPOCHS

In [ ]:
train_data, val_data = SpamDatasetBert(df_train), SpamDatasetBert(df_val)

In [ ]:
EPOCHS = 3
model = BertClassifier()
LR = 1e-4

train(model, train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [24:20<00:00,  2.08it/s]


Epochs: 1 | Train Loss:  0.041 | Train Accuracy:  0.566 | Val Loss:  0.044 | Val Accuracy:  0.530


100%|██████████| 3033/3033 [24:19<00:00,  2.08it/s]


Epochs: 2 | Train Loss:  0.043 | Train Accuracy:  0.515 | Val Loss:  0.043 | Val Accuracy:  0.533


100%|██████████| 3033/3033 [24:18<00:00,  2.08it/s]


Epochs: 3 | Train Loss:  0.043 | Train Accuracy:  0.521 | Val Loss:  0.043 | Val Accuracy:  0.535


In [ ]:
EPOCHS = 3
model = BertLstmClassifier()
LR = 1e-4

train(model, train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [24:24<00:00,  2.07it/s]


Epochs: 1 | Train Loss:  0.042 | Train Accuracy:  0.544 | Val Loss:  0.043 | Val Accuracy:  0.535


100%|██████████| 3033/3033 [24:24<00:00,  2.07it/s]


Epochs: 2 | Train Loss:  0.043 | Train Accuracy:  0.531 | Val Loss:  0.043 | Val Accuracy:  0.535


100%|██████████| 3033/3033 [24:27<00:00,  2.07it/s]


Epochs: 3 | Train Loss:  0.043 | Train Accuracy:  0.530 | Val Loss:  0.043 | Val Accuracy:  0.535


In [ ]:
EPOCHS = 3
model = BertCNNLstmClassifier()
LR = 1e-4

train(model, train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [26:42<00:00,  1.89it/s]


Epochs: 1 | Train Loss:  0.042 | Train Accuracy:  0.541 | Val Loss:  0.043 | Val Accuracy:  0.534


100%|██████████| 3033/3033 [26:37<00:00,  1.90it/s]


Epochs: 2 | Train Loss:  0.043 | Train Accuracy:  0.529 | Val Loss:  0.043 | Val Accuracy:  0.536


100%|██████████| 3033/3033 [26:37<00:00,  1.90it/s]


Epochs: 3 | Train Loss:  0.043 | Train Accuracy:  0.528 | Val Loss:  0.043 | Val Accuracy:  0.535


In [ ]:
EPOCHS = 3
model = BertCNNClassifier()
LR = 1e-4

train(model, train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [24:26<00:00,  2.07it/s]


Epochs: 1 | Train Loss:  0.039 | Train Accuracy:  0.584 | Val Loss:  0.044 | Val Accuracy:  0.515


100%|██████████| 3033/3033 [24:23<00:00,  2.07it/s]


Epochs: 2 | Train Loss:  0.044 | Train Accuracy:  0.513 | Val Loss:  0.043 | Val Accuracy:  0.517


100%|██████████| 3033/3033 [24:23<00:00,  2.07it/s]


Epochs: 3 | Train Loss:  0.043 | Train Accuracy:  0.517 | Val Loss:  0.043 | Val Accuracy:  0.535


### GPT Models - 1 EPOCHS

In [ ]:
train_data, val_data = SpamDatasetGpt(df_train), SpamDatasetGpt(df_val)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
EPOCHS = 1
model = GPT2Classifier()
LR = 1e-4

train(model, train_data, val_data, LR, EPOCHS, batch_size=8)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

100%|██████████| 6066/6066 [32:21<00:00,  3.12it/s]


Epochs: 1 | Train Loss:  0.005 | Train Accuracy:  0.987 | Val Loss:  0.002 | Val Accuracy:  0.994


In [ ]:
EPOCHS = 1
model = GPT2CNNClassifier()
LR = 1e-4

train(model, train_data, val_data, LR, EPOCHS, batch_size=8)

100%|██████████| 6066/6066 [31:56<00:00,  3.17it/s]


Epochs: 1 | Train Loss:  0.005 | Train Accuracy:  0.986 | Val Loss:  0.002 | Val Accuracy:  0.995


In [ ]:
EPOCHS = 1
model = GPT2LstmClassifier()
LR = 1e-4

train(model, train_data, val_data, LR, EPOCHS, batch_size=8)

100%|██████████| 6066/6066 [31:58<00:00,  3.16it/s]


Epochs: 1 | Train Loss:  0.012 | Train Accuracy:  0.959 | Val Loss:  0.003 | Val Accuracy:  0.994


In [ ]:
EPOCHS = 1
model = GPT2CNNLstmClassifier()
LR = 1e-4

train(model, train_data, val_data, LR, EPOCHS, batch_size=8)

100%|██████████| 6066/6066 [32:04<00:00,  3.15it/s]


Epochs: 1 | Train Loss:  0.007 | Train Accuracy:  0.981 | Val Loss:  0.003 | Val Accuracy:  0.994


### GPT Modes - 3 EPOCHS

In [ ]:
train_data, val_data = SpamDatasetGpt(df_train), SpamDatasetGpt(df_val)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
EPOCHS = 3
model = GPT2Classifier()
LR = 1e-4

train(model, train_data, val_data, LR, EPOCHS, batch_size=8)

100%|██████████| 6066/6066 [31:50<00:00,  3.17it/s]


Epochs: 1 | Train Loss:  0.006 | Train Accuracy:  0.986 | Val Loss:  0.002 | Val Accuracy:  0.995


100%|██████████| 6066/6066 [31:51<00:00,  3.17it/s]


Epochs: 2 | Train Loss:  0.002 | Train Accuracy:  0.995 | Val Loss:  0.003 | Val Accuracy:  0.992


100%|██████████| 6066/6066 [31:50<00:00,  3.18it/s]


Epochs: 3 | Train Loss:  0.001 | Train Accuracy:  0.998 | Val Loss:  0.002 | Val Accuracy:  0.994


In [ ]:
EPOCHS = 3
model = GPT2CNNClassifier()
LR = 1e-4

train(model, train_data, val_data, LR, EPOCHS, batch_size=8)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

100%|██████████| 6066/6066 [30:54<00:00,  3.27it/s]


Epochs: 1 | Train Loss:  0.005 | Train Accuracy:  0.986 | Val Loss:  0.002 | Val Accuracy:  0.995


100%|██████████| 6066/6066 [30:59<00:00,  3.26it/s]


Epochs: 2 | Train Loss:  0.002 | Train Accuracy:  0.996 | Val Loss:  0.003 | Val Accuracy:  0.993


100%|██████████| 6066/6066 [30:54<00:00,  3.27it/s]


Epochs: 3 | Train Loss:  0.001 | Train Accuracy:  0.998 | Val Loss:  0.003 | Val Accuracy:  0.994


In [ ]:
EPOCHS = 3
model = GPT2LstmClassifier()
LR = 1e-4

train(model, train_data, val_data, LR, EPOCHS, batch_size=8)

100%|██████████| 6066/6066 [31:14<00:00,  3.24it/s]


Epochs: 1 | Train Loss:  0.011 | Train Accuracy:  0.968 | Val Loss:  0.005 | Val Accuracy:  0.985


100%|██████████| 6066/6066 [30:57<00:00,  3.27it/s]


Epochs: 2 | Train Loss:  0.002 | Train Accuracy:  0.995 | Val Loss:  0.002 | Val Accuracy:  0.994


100%|██████████| 6066/6066 [30:54<00:00,  3.27it/s]


Epochs: 3 | Train Loss:  0.001 | Train Accuracy:  0.997 | Val Loss:  0.002 | Val Accuracy:  0.995


In [ ]:
EPOCHS = 3
model = GPT2CNNLstmClassifier()
LR = 1e-4

train(model, train_data, val_data, LR, EPOCHS, batch_size=8)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

100%|██████████| 6066/6066 [31:45<00:00,  3.18it/s]


Epochs: 1 | Train Loss:  0.013 | Train Accuracy:  0.963 | Val Loss:  0.004 | Val Accuracy:  0.988


100%|██████████| 6066/6066 [31:45<00:00,  3.18it/s]


Epochs: 2 | Train Loss:  0.002 | Train Accuracy:  0.994 | Val Loss:  0.003 | Val Accuracy:  0.993


100%|██████████| 6066/6066 [31:46<00:00,  3.18it/s]


Epochs: 3 | Train Loss:  0.001 | Train Accuracy:  0.998 | Val Loss:  0.002 | Val Accuracy:  0.995


## SHAP

In [ ]:
!pip install shap
import shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.7/535.7 kB 3.2 MB/s eta 0:00:00


In [ ]:
def f(texts):
    tokens = [tokenizer(text, padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in texts]
    ids = torch.concat([x['input_ids'] for x in tokens]).cuda()
    masks = torch.concat([x['attention_mask'] for x in tokens]).cuda()
    outputs = model(ids, masks).detach().cpu().numpy()
    return outputs.ravel()

### Convetional

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
train_data, val_data = SpamDatasetBert(df_train), SpamDatasetBert(df_val)
EPOCHS = 3
model = CnnClassifier()
LR = 1e-4
train(model,train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [00:40<00:00, 75.27it/s]


Epochs: 1 | Train Loss:  0.014 | Train Accuracy:  0.904 | Val Loss:  0.007 | Val Accuracy:  0.957


100%|██████████| 3033/3033 [00:40<00:00, 74.61it/s]


Epochs: 2 | Train Loss:  0.006 | Train Accuracy:  0.969 | Val Loss:  0.005 | Val Accuracy:  0.971


100%|██████████| 3033/3033 [00:40<00:00, 74.78it/s]


Epochs: 3 | Train Loss:  0.004 | Train Accuracy:  0.976 | Val Loss:  0.004 | Val Accuracy:  0.977


In [ ]:
explainer = shap.Explainer(f, tokenizer)

In [ ]:
shap_values = explainer(df_val['email_content'].iloc[[100]].tolist())
shap.plots.text(shap_values)

### BERT


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
train_data, val_data = SpamDatasetBert(df_train), SpamDatasetBert(df_val)
EPOCHS = 3
model = BertClassifier()
LR = 1e-4
train(model,train_data, val_data, LR, EPOCHS)

100%|██████████| 3033/3033 [24:25<00:00,  2.07it/s]


Epochs: 1 | Train Loss:  0.036 | Train Accuracy:  0.611 | Val Loss:  0.045 | Val Accuracy:  0.494


100%|██████████| 3033/3033 [24:25<00:00,  2.07it/s]


Epochs: 2 | Train Loss:  0.044 | Train Accuracy:  0.515 | Val Loss:  0.043 | Val Accuracy:  0.531


100%|██████████| 3033/3033 [24:25<00:00,  2.07it/s]


Epochs: 3 | Train Loss:  0.043 | Train Accuracy:  0.515 | Val Loss:  0.043 | Val Accuracy:  0.530


In [ ]:
explainer = shap.Explainer(f, tokenizer)

In [ ]:
shap_values = explainer(df_val['email_content'].iloc[[100]].tolist())
shap.plots.text(shap_values)

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:11, 11.04s/it]               


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
train_data, val_data = SpamDatasetBert(df_train.head(1000)), SpamDatasetBert(df_val)
EPOCHS = 1
model = BertClassifier()
LR = 1e-4
train(model,train_data, val_data, LR, EPOCHS)

100%|██████████| 63/63 [00:30<00:00,  2.09it/s]


Epochs: 1 | Train Loss:  0.037 | Train Accuracy:  0.674 | Val Loss:  0.022 | Val Accuracy:  0.864


### GPT


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path='gpt2')
# Set the EOS token as the pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
train_data, val_data = SpamDatasetGpt(df_train), SpamDatasetGpt(df_val)
EPOCHS = 3
model = GPT2Classifier()
LR = 1e-4
train(model,train_data, val_data, LR, EPOCHS)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Epochs: 1 | Train Loss:  0.003 | Train Accuracy:  0.986 | Val Loss:  0.002 | Val Accuracy:  0.986


100%|██████████| 3033/3033 [28:10<00:00,  1.79it/s]


Epochs: 2 | Train Loss:  0.001 | Train Accuracy:  0.997 | Val Loss:  0.002 | Val Accuracy:  0.993


100%|██████████| 3033/3033 [28:12<00:00,  1.79it/s]


Epochs: 3 | Train Loss:  0.000 | Train Accuracy:  0.998 | Val Loss:  0.001 | Val Accuracy:  0.996


In [ ]:
EPOCHS = 1
model = GPT2Classifier()
LR = 1e-4

train(model, train_data, val_data, LR, EPOCHS, batch_size=8)

100%|██████████| 6066/6066 [31:24<00:00,  3.22it/s]


Epochs: 1 | Train Loss:  0.005 | Train Accuracy:  0.987 | Val Loss:  0.002 | Val Accuracy:  0.996


In [ ]:
def f(texts):
    tokens = [tokenizer(text, padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in texts]
    ids = torch.concat([x['input_ids'] for x in tokens]).cuda()
    masks = torch.concat([x['attention_mask'] for x in tokens]).cuda()
    outputs = model(ids, masks).detach().cpu().numpy()
    return outputs.ravel()

In [ ]:
explainer = shap.Explainer(f, tokenizer)

In [ ]:
shap_values = explainer(df_val['email_content'].iloc[[100]].tolist())

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:17, 17.31s/it]               


In [ ]:
shap.plots.text(shap_values)

# preliminary test results


In [ ]:
#LstmClassifier mask results
#100%|██████████| 2427/2427 [01:22<00:00, 29.52it/s]
#Epochs: 1 | Train Loss:  0.029 | Train Accuracy:  0.701 | Val Loss:  0.026 | Val Accuracy:  0.747

In [ ]:
# LstmClassifier non-mask results
# 100%|██████████| 2427/2427 [01:23<00:00, 28.93it/s]
# Epochs: 1 | Train Loss:  0.029 | Train Accuracy:  0.691 | Val Loss:  0.034 | Val Accuracy:  0.582

In [ ]:
!nvidia-smi

In [ ]:
# EPOCHS = 1
# model = BertLstmClassifier()
# # model = BertCNNClassifier()
# # model = BertClassifier()
# # model = BertCNNLstmClassifier()
# # model3 = GPT2LstmClassifier()
# # model2 = GPT2CNNClassifier()
# # model1 = GPT2Classifier()
# # model4 = GPT2CNNLstmClassifier()
# LR = 1e-4

# train(model, df_train, df_val, LR, EPOCHS)